In [1]:
"""
This script transform the data files into a standard format (columns,names,order etc.).
In the feature_engineering script you only generate features and not change format.
It has to be that way so that when we want to generate a feature matrix from the test data we'll need only 
to refomatted it according to the train format and then use the feature engineering script as is.
For example, if we take the 5 most popular genre as onehot columns, they could be different in train and test, therefore,
we'll need to adjust the test according to the train and continue from there.
"""

# TODO: Add external data sources
# Kaggle original data - need to populate it with vote count/avg (https://github.com/jasonliu119/movie_predict)
                        # Integrate it 
                        # check for duplications
                        # fix fields (in all data)

"\nThis script transform the data files into a standard format (columns,names,order etc.).\nIn the feature_engineering script you only generate features and not change format.\nIt has to be that way so that when we want to generate a feature matrix from the test data we'll need only \nto refomatted it according to the train format and then use the feature engineering script as is.\nFor example, if we take the 5 most popular genre as onehot columns, they could be different in train and test, therefore,\nwe'll need to adjust the test according to the train and continue from there.\n"

In [1]:
import pandas as pd
import numpy as np 
import json
import ast
from collections import Counter
# pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

In [2]:
DATA_FOLDER = "/StudentData/hw1/"

In [3]:
train = pd.read_csv(DATA_FOLDER + "train.tsv",sep="\t")
test = pd.read_csv(DATA_FOLDER + "test.tsv",sep="\t")
test.head()
one_hot_columns = {}

In [4]:
# fixing fields from external source
dataset = pd.concat([train, test], ignore_index=True, sort=True)
train.shape, test.shape, dataset.shape

((5215, 27), (1738, 27), (6953, 27))

In [5]:
def get_values(elem_str,field):
    elem_list = [] if pd.isna(elem_str) else ast.literal_eval(elem_str)
    if isinstance(elem_list, dict):
        elem_list = [elem_list]
    values = []
    for elem in elem_list:
        values.append(elem[field])
    return values


def json_to_onehot(df,col,field,cutoff_cretria):
    column_format = lambda val : col+"_"+str(val)
    possible_values = Counter()
    df[col].apply(lambda elem_str : possible_values.update(get_values(elem_str,field)))
    possible_values = sorted(cutoff_cretria(possible_values))
    one_hot_columns[col] = possible_values
    new_columns = [column_format(val) for val in possible_values]
    for new_col in new_columns:
        df[new_col] = 0
    df[new_columns].replace(np.nan, 0, inplace=True)
    print(col," added ",len(new_columns), " columns")
#     print(col,field,possible_values)
    for i,row in df.iterrows():
        for val in [elem for elem in get_values(row[col],field) if elem in possible_values]:
            df.loc[i,column_format(val)] = 1


# def add_one_hot(df,col,cutoff_cretria):
#     possible_values = Counter(df[col])
#     possible_values = sorted(cutoff_cretria(possible_values))
            
            
def reformat_data(df_input,max_number_of_columns,threshold_cutoff):  
    df = df_input.copy()
    # adding one hot columns only if the number of samples reaches the threshold
    cutoff_cretria = lambda possible_values : [field_value for field_value, num_samples in possible_values.most_common(max_number_of_columns) if num_samples>=threshold_cutoff]
    json_to_onehot(df,"belongs_to_collection","name",cutoff_cretria)
    json_to_onehot(df,"genres","name",cutoff_cretria)
    json_to_onehot(df,"production_companies","id",cutoff_cretria)
    json_to_onehot(df,"production_countries","name",cutoff_cretria)
    json_to_onehot(df,"spoken_languages","iso_639_1",cutoff_cretria)
    json_to_onehot(df,"cast","name",cutoff_cretria)
    json_to_onehot(df,"crew","name",cutoff_cretria)
#     one_hot_lang = pd.get_dummies(df['original_language'],prefix='original_language')
#     df = df.join(one_hot_lang)
#     chosen_columns = ['budget','homepage', 'original_title', 'overview', 'popularity', 'release_date', 'revenue', 'runtime', 'status', 'tagline', 'title', 'video', 'vote_average', 'vote_count']
#     add_columns = df.columns[len(df_input.columns):]
#     df = df[list(chosen_columns) + list(add_columns)]
    return df 

In [6]:
# train_reformated = reformat_data(train,20,5)
# train_reformated.head()
dataset_reformated = reformat_data(dataset,20,5)
dataset_reformated

/home/student/miniconda3/envs/hw1/lib/python3.7/site-packages/pandas/core/frame.py:4172: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  method=method,


belongs_to_collection  added  20  columns
genres  added  19  columns
production_companies  added  20  columns
production_countries  added  20  columns
spoken_languages  added  20  columns
cast  added  20  columns
crew  added  20  columns


,Keywords,backdrop_path,belongs_to_collection,budget,cast,crew,genres,homepage,id,imdb_id,original_language,original_title,overview,popularity,poster_path,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count,belongs_to_collection_A Nightmare on Elm Street Collection,belongs_to_collection_Child's Play Collection,belongs_to_collection_Die Hard Collection,belongs_to_collection_Friday the 13th Collection,belongs_to_collection_Halloween Collection,belongs_to_collection_Harry Potter Collection,belongs_to_collection_Ice Age Collection,belongs_to_collection_James Bond Collection,belongs_to_collection_One Piece Collection,belongs_to_collection_Paranormal Activity Collection,belongs_to_collection_Pirates of the Caribbean Collection,belongs_to_collection_Pokémon Collection,belongs_to_collection_Police Academy Collection,belongs_to_collection_Rocky Collection,belongs_to_collection_Saw Collection,belongs_to_collection_Star Trek: The Original Series Collection,belongs_to_collection_The Exorcist Collection,belongs_to_collection_The Muppet Collection,belongs_to_collection_The Pink Panther (Original) Collection,belongs_to_collection_The Terminator Collection,genres_Action,genres_Adventure,genres_Animation,genres_Comedy,genres_Crime,genres_Documentary,genres_Drama,genres_Family,genres_Fantasy,genres_History,genres_Horror,genres_Music,genres_Mystery,genres_Romance,genres_Science Fiction,genres_TV Movie,genres_Thriller,genres_War,genres_Western,production_companies_2,production_companies_4,production_companies_5,production_companies_7,production_companies_12,production_companies_14,production_companies_21,production_companies_25,production_companies_33,production_companies_34,production_companies_60,production_companies_79,production_companies_104,production_companies_174,production_companies_508,production_companies_559,production_companies_694,production_companies_1632,production_companies_7295,production_companies_9195,production_countries_Australia,production_countries_Belgium,production_countries_Canada,production_countries_China,production_countries_Denmark,production_countries_France,production_countries_Germany,production_countries_Hong Kong,production_countries_India,production_countries_Ireland,production_countries_Italy,production_countries_Japan,production_countries_Mexico,production_countries_Netherlands,production_countries_Russia,production_countries_South Korea,production_countries_Spain,production_countries_Sweden,production_countries_United Kingdom,production_countries_United States of America,spoken_languages_ar,spoken_languages_cn,spoken_languages_cs,spoken_languages_de,spoken_languages_en,spoken_languages_es,spoken_languages_fr,spoken_languages_he,spoken_languages_hi,spoken_languages_hu,spoken_languages_it,spoken_languages_ja,spoken_languages_ko,spoken_languages_la,spoken_languages_pl,spoken_languages_pt,spoken_languages_ru,spoken_languages_sv,spoken_languages_tr,spoken_languages_zh,cast_Alec Baldwin,cast_Bruce Willis,cast_Dennis Quaid,cast_Frank Welker,cast_J.K. Simmons,cast_John Goodman,cast_John Leguizamo,cast_Johnny Depp,cast_Keith David,cast_Liam Neeson,cast_Matt Damon,cast_Morgan Freeman,cast_Nicolas Cage,cast_Richard Jenkins,cast_Robert De Niro,cast_Robin Williams,cast_Samuel L. Jackson,cast_Steve Buscemi,cast_Sylvester Stallone,cast_Willem Dafoe,crew_Alan Silvestri,crew_Avy Kaufman,crew_Barbara Harris,crew_Bob Weinstein,crew_Dan O'Connell,crew_Dan Perri,crew_Deborah Aquila,crew_Gary Burritt,crew_Hans Zimmer,crew_Harvey Weinstein,crew_James Horner,crew_James Newton Howard,crew_John T. Cucci,crew_Kerry Barden,crew_Mary Vernieu,crew_Mo Henry,crew_Nerses Gezalyan,crew_Robert Rodriguez,crew_Steven Spielberg,crew_Tricia Wood
0,"[{'id': 697, 'name': 'loss of loved one'}, {'i...",/7IBpOrw0ATwL1AOV97mtsceDpYs.jpg,"{'id': 556, 'name': 'Spider-Man Collection', '...",258000000,"[{'cast_id': 30, 'character': 'Peter Parker / ...","[{'credit_id': '

In [7]:
dataset_reformated.to_csv(DATA_FOLDER + "dataset_reformated.csv",index=False)

In [8]:
# import pickle
# with open(DATA_FOLDER + "one_hot_columns", 'wb') as file:
#     pickle.dump(one_hot_columns, file)

In [9]:
# Counter(dataset["original_language"]).most_common(20)

[('en', 5802),
 ('fr', 208),
 ('hi', 161),
 ('ja', 110),
 ('es', 97),
 ('ru', 89),
 ('it', 78),
 ('ko', 68),
 ('zh', 54),
 ('de', 50),
 ('cn', 47),
 ('ta', 21),
 ('tr', 20),
 ('ml', 18),
 ('pt', 16),
 ('sv', 15),
 ('da', 12),
 ('ar', 9),
 ('te', 8),
 ('fa', 7)]

In [8]:
one_hot_columns

{'belongs_to_collection': ['A Nightmare on Elm Street Collection',
  "Child's Play Collection",
  'Die Hard Collection',
  'Friday the 13th Collection',
  'Halloween Collection',
  'Harry Potter Collection',
  'Ice Age Collection',
  'James Bond Collection',
  'One Piece Collection',
  'Paranormal Activity Collection',
  'Pirates of the Caribbean Collection',
  'Pokémon Collection',
  'Police Academy Collection',
  'Rocky Collection',
  'Saw Collection',
  'Star Trek: The Original Series Collection',
  'The Exorcist Collection',
  'The Muppet Collection',
  'The Pink Panther (Original) Collection',
  'The Terminator Collection'],
 'genres': ['Action',
  'Adventure',
  'Animation',
  'Comedy',
  'Crime',
  'Documentary',
  'Drama',
  'Family',
  'Fantasy',
  'History',
  'Horror',
  'Music',
  'Mystery',
  'Romance',
  'Science Fiction',
  'TV Movie',
  'Thriller',
  'War',
  'Western'],
 'production_companies': [2,
  4,
  5,
  7,
  12,
  14,
  21,
  25,
  33,
  34,
  60,
  79,
  104,
 